# INF-395 Redes Neuronales Artificiales II-2018

## Tarea 2 - Redes Convolucionales y sus aplicaciones


26/11/2018

* Rodrigo González Smith 201303026-2
* Ignacio Valenzuela Albornoz

## Introducción

In [3]:
import numpy as np   
import os   
from keras.datasets import cifar10
import keras
import matplotlib.pyplot as plt
%inline matplotlib

UsageError: Line magic function `%inline` not found.


## 1. Convolutional Neural Networks (CNN) en CIFAR

### 1.a)

In [1]:
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']  
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Using TensorFlow backend.
/home/ipresnya/aidPython/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Separamos el set de train para tener uno de validación también:

In [7]:
n_train = len(x_train)
x_val = x_train[int(0.8*n_train):n_train]
y_val = y_train[int(0.8*n_train):n_train]
x_train = x_train[:int(0.8*n_train)]
y_train = y_train[:int(0.8*n_train)]